In [2]:
import nibabel as nib
import numpy as np
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.padding import PKCS7
import os
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
import logging
from nibabel.filebasedimages import FileBasedImage
import cv2
import pandas as pd
from pyspark.rdd import RDD
from io import BytesIO
logging.basicConfig(level=logging.INFO)

logger = logging.getLogger(__name__)

In [2]:
import zipfile
import os

# Danh sách các file ZIP
zip_files = [
    "ct_scans.zip",
    "infection_mask.zip",
    "lung_and_infection_mask.zip",
    "lung_mask.zip",
]

# Thư mục tổng để chứa các thư mục con
output_folder = "data"

# Tạo thư mục tổng nếu chưa tồn tại
os.makedirs(output_folder, exist_ok=True)

# Giải nén từng file ZIP vào thư mục riêng
for zip_file in zip_files:
    try:
        # Lấy tên file ZIP (không có phần mở rộng) để đặt tên thư mục
        folder_name = os.path.splitext(os.path.basename(zip_file))[0]
        extract_path = os.path.join(output_folder, folder_name)
        
        # Tạo thư mục riêng cho từng file ZIP
        os.makedirs(extract_path, exist_ok=True)
        
        # Giải nén file ZIP vào thư mục riêng
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            print(f"Đang giải nén: {zip_file} vào {extract_path}")
            zip_ref.extractall(extract_path)
        
        print(f"Giải nén thành công: {zip_file}")
    except zipfile.BadZipFile:
        print(f"Lỗi: {zip_file} không phải là file ZIP hợp lệ")


o1Qzq9W0RuSODfOdOWMO
NvuyLbJngKDNtNf0GdllPvbtZk3Wgg2SbgOaIqLF


In [3]:
ct_nii: FileBasedImage = nib.load(f"{CT_SCANS}/coronacases_org_001.nii")
header = ct_nii.header
affine = ct_nii.affine
data = ct_nii.get_fdata()
data = np.array(data)
data[0], np.min(data), np.max(data), data.shape

(array([[-998., -998., -999., ..., -998., -998., -998.],
        [-998., -998., -999., ..., -998., -998., -998.],
        [-998., -998., -999., ..., -998., -998., -998.],
        ...,
        [-998., -999., -998., ..., -998., -998., -998.],
        [-998., -999., -998., ..., -998., -998., -998.],
        [-998., -999., -998., ..., -998., -998., -998.]]),
 np.float64(-1021.0),
 np.float64(2996.0),
 (512, 512, 301))

In [4]:
spark = SparkSession.builder \
    .appName("NIfTI_Processing") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .config("spark.driver.maxResultSize", "10g") \
    .config("spark.hadoop.fs.s3a.access.key", S3_ACCESS_KEY) \
    .config("spark.hadoop.fs.s3a.secret.key", S3_SECRET_KEY) \
    .config("spark.hadoop.fs.s3a.endpoint", "https://s3.captechvn.com") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .getOrCreate()
sc = spark.sparkContext

24/12/13 16:20:48 WARN Utils: Your hostname, SerVer resolves to a loopback address: 127.0.1.1; using 192.168.1.250 instead (on interface enp3s0f0)
24/12/13 16:20:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/13 16:20:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Load into RDDs

In [5]:
##### Extract #####
def download_data():
    if os.path.exists(OUTPUT_DIR):
        return
    os.makedirs("OUTPUT_DIR", exist_ok=True)
    # Download latest version to rawdata folder
    path = kagglehub.dataset_download("andrewmvd/covid19-ct-scans")
    print("Path to dataset files:", path)
    # Move the downloaded folder to the rawdata directory, For example: 4/ct_scans, 4/metadata -> rawdata/ct_scans, rawdata/metadata
    os.rename(path, OUTPUT_DIR)
    print("Data downloaded and unzipped successfully!")

def extract_metadata():
    # Extract metadata from the downloaded dataset
    path = os.path.join(OUTPUT_DIR, "metadata.csv")
    metadata = pd.read_csv(path)
    logger.info("Metadata extracted successfully!")
    metadata.replace("../input/covid19-ct-scans/", "", regex=True, inplace=True)
    return metadata

def extract_image_data(metadata: pd.DataFrame) -> RDD:
    # Convert DataFrame to np array
    image_paths: np.ndarray = metadata.to_numpy().flatten()
    # Extract image data
    def load_image(x):
        path = os.path.join(OUTPUT_DIR, x)
        nifti_data = nib.load(path)
        header = nifti_data.header
        affine = nifti_data.affine
        image_data = nifti_data.get_fdata()
        image_data = np.array(image_data)

        return image_data, header, affine, x

    image_rdds = sc.parallelize(image_paths).map(load_image)

    return image_rdds

In [6]:
def normalize_and_resize_nii(file_path, target_shape=(256, 256, 128)):
    """
    Chuẩn hóa và resize ảnh .nii về kích thước cố định.
    """
    # read file .nii
    img = nib.load(file_path)
    data = img.get_fdata()

    # Transforming data to float32 type
    data = data.astype(np.float32)

    # Get the original shape
    original_shape = data.shape

    # Create an array for target shape
    resized_data = np.zeros(target_shape, dtype=np.float32)

    #  crop or pad images
    min_shape = np.minimum(original_shape, target_shape)
    crop_start = [(o - m) // 2 for o, m in zip(original_shape, min_shape)]
    crop_end = [start + m for start, m in zip(crop_start, min_shape)]
    pad_start = [(t - m) // 2 for t, m in zip(target_shape, min_shape)]
    pad_end = [start + m for start, m in zip(pad_start, min_shape)]

    #(crop)
    slices = tuple(slice(start, end) for start, end in zip(crop_start, crop_end))
    cropped_data = data[slices]

    # Padding for getting the target shape
    resized_data[tuple(slice(start, end) for start, end in zip(pad_start, pad_end))] = cropped_data

    # create new file .nii with the data which are resized
    resized_img = nib.Nifti1Image(resized_data, img.affine, img.header)

    return resized_img


In [8]:
download_data()
metadata = extract_metadata()
rdds = extract_image_data(metadata)
rdds = resize_image(rdds)
rdds = normalize_image(rdds)
encrypted = encrypt_image(rdds)
# save_data(encrypted)
save_to_bucket(encrypted)
# save_encrypted_image(encrypted)

INFO:__main__:Metadata extracted successfully!
INFO:__main__:Image resized successfully!
INFO:__main__:Image normalized successfully!
INFO:__main__:Image encrypted successfully!
/home/aphuc/web/data_science/pytorch-notebook/ETL-MI/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 's3.captechvn.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/aphuc/web/data_science/pytorch-notebook/ETL-MI/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 's3.captechvn.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/aphuc/web/data_science/pytorch-notebook/ETL-MI/.venv/lib/python3.12/site-packages/urllib3/connec

In [10]:
import nibabel as nib
import os
import numpy as np
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.padding import PKCS7
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend

# Folders which include file .nii which are normalized
INPUT_FOLDERS = ["/content/drive/MyDrive/data-bigdatatechnlogy/BigData-project/data/Final_normalized_data/ct_scans",
                 "/content/drive/MyDrive/data-bigdatatechnlogy/BigData-project/data/Final_normalized_data/infection_mask",
                 "/content/drive/MyDrive/data-bigdatatechnlogy/BigData-project/data/Final_normalized_data/lung_and_infection_mask",
                 "/content/drive/MyDrive/data-bigdatatechnlogy/BigData-project/data/Final_normalized_data/lung_mask"]
OUTPUT_DIR = "/content/drive/MyDrive/data-bigdatatechnlogy/BigData-project/data-encrypt"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Encrypt function:
def encrypt_data(data, password):
    # convert data to bytes
    data_bytes = data.tobytes()

    # salt and vector (IV)
    salt = os.urandom(16)
    iv = os.urandom(16)

    # create encrypt key:
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    key = kdf.derive(password.encode())  # create encrypt key from the password

    # Set AES encryption algorithm with CBC mode
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()

    # Padding data to ensure correct AES block size
    padder = PKCS7(algorithms.AES.block_size).padder()
    padded_data = padder.update(data_bytes) + padder.finalize()

    # Encrypt data
    encrypted_data = encryptor.update(padded_data) + encryptor.finalize()
    return salt, iv, encrypted_data

# Encrypt all normalized files from folders
password = "xemlamgi"  # password

for folder_idx, folder in enumerate(INPUT_FOLDERS):
    # Create a folder to save the corresponding encrypted file
    encrypted_folder_path = os.path.join(OUTPUT_DIR, f"encrypted_{os.path.basename(folder)}")
    os.makedirs(encrypted_folder_path, exist_ok=True)

    print(f"Đang xử lý thư mục chuẩn hóa: {folder}")

    # Browse all files in the current directory
    for filename in os.listdir(folder):
        if filename.endswith('.nii'):
            filepath = os.path.join(folder, filename)

            # Read file .nii
            img = nib.load(filepath)
            data = img.get_fdata()

            # Encrypt data
            salt, iv, encrypted_data = encrypt_data(data, password)

            # Save the encrypted data to a binary file in the results directory.
            output_file = os.path.join(encrypted_folder_path, f"{os.path.splitext(filename)[0]}.enc")
            with open(output_file, 'wb') as f:
                f.write(salt)  # salt
                f.write(iv)    # IV
                # Write the size (target_shape) as int32
                target_shape = data.shape
                f.write(np.array(target_shape, dtype=np.int32).tobytes())
                f.write(encrypted_data)  # write encrypted data

            print(f"Đã mã hóa và lưu file: {output_file}")


In [16]:
import nibabel as nib
import os
import numpy as np
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.padding import PKCS7
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend

# The folder contains 4 encrypted folders and the resulting merge folder.
BASE_ENCRYPTED_DIR = "/content/drive/MyDrive/data-bigdatatechnlogy/BigData-project/data-encrypt"
BASE_DECRYPTED_DIR = "/content/drive/MyDrive/data-bigdatatechnlogy/BigData-project/decrypted_data"
os.makedirs(BASE_DECRYPTED_DIR, exist_ok=True)

# List of subfolder names
FOLDERS = ["/content/drive/MyDrive/data-bigdatatechnlogy/BigData-project/data-encrypt/encrypted_ct_scans",
           "/content/drive/MyDrive/data-bigdatatechnlogy/BigData-project/data-encrypt/encrypted_infection_mask",
           "/content/drive/MyDrive/data-bigdatatechnlogy/BigData-project/data-encrypt/encrypted_lung_and_infection_mask",
           "/content/drive/MyDrive/data-bigdatatechnlogy/BigData-project/data-encrypt/encrypted_lung_mask"]

# Descryption fucntion
def decrypt_data(encrypted_data, salt, iv, password):
    # create key from password
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    key = kdf.derive(password.encode())

    # Set AES algorithm with CBC mode
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    decryptor = cipher.decryptor()

    # descrypt data
    padded_data = decryptor.update(encrypted_data) + decryptor.finalize()

    # Remove padding to get original data
    unpadder = PKCS7(algorithms.AES.block_size).unpadder()
    data = unpadder.update(padded_data) + unpadder.finalize()

    # Chuyển dữ liệu từ bytes về mảng NumPy
    return np.frombuffer(data, dtype=np.float64)

# descrypt password
password = "xemlamgi"

# Loop through all encrypted folders
for folder in FOLDERS:
    decrypted_folder_name = os.path.basename(folder)  # Get the last directory name
    decrypted_folder_path = os.path.join(BASE_DECRYPTED_DIR, decrypted_folder_name)
    os.makedirs(decrypted_folder_path, exist_ok=True)  # Create corresponding decryption folder

    # Decrypt all files in the current folder
    for filename in os.listdir(folder):
        if filename.endswith('.enc'):
            filepath = os.path.join(folder, filename)

            # Read encrypted data from file
            with open(filepath, 'rb') as f:
                salt = f.read(16)  # salt (16 bytes)
                iv = f.read(16)    # IV (16 bytes)
                target_shape = np.frombuffer(f.read(12), dtype=np.int32)
                encrypted_data = f.read()  # Đọc dữ liệu mã hóa còn lại

            # descrypt data
            decrypted_data = decrypt_data(encrypted_data, salt, iv, password)

            # Convert data to original size
            reshaped_data = decrypted_data.reshape(target_shape)

            # Save decrypted data to .nii file
            output_file = os.path.join(decrypted_folder_path, filename.replace('.enc', '.nii'))
            nib.save(nib.Nifti1Image(reshaped_data, affine=np.eye(4)), output_file)

            print(f"Giải mã và lưu file: {output_file}")

print("\nHoàn thành giải mã tất cả dữ liệu.")


In [ ]:
import os
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt

def window_image(image, window_center, window_width):
    """
    Using window level and window width for images:
    """
    min_val = window_center - (window_width / 2)
    max_val = window_center + (window_width / 2)
    windowed_image = np.clip(image, min_val, max_val)
    normalized_image = (windowed_image - min_val) / (max_val - min_val) * 255.0
    return normalized_image.astype(np.uint8)

def calculate_grid(num_images):
    """
    Calculate the optimal number of rows and columns based on the number of images.
    """
    rows = int(np.ceil(np.sqrt(num_images)))
    cols = int(np.ceil(num_images / rows))
    return rows, cols

def show_multiple_images_v2(file_paths, window_center, window_width):
    """
    Display multiple CT images from file list with automatic grid.
    """
    num_images = len(file_paths)
    if num_images == 0:
        print("Không có ảnh nào để hiển thị.")
        return

    # Optimal grid calculation
    rows, cols = calculate_grid(num_images)

    # Create charts
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 4, rows * 4))
    axes = np.array(axes).flatten()  # Convert axes to flat list

    for i, file_path in enumerate(file_paths):
        # Load images from NIfTI file
        img = nib.load(file_path)
        data = img.get_fdata()

        # Select the middle Axial slice
        axial_slice_idx = data.shape[2] // 2
        axial_slice = data[:, :, axial_slice_idx]

        # Apply windowing
        processed_image = window_image(axial_slice, window_center, window_width)

        # Show images
        axes[i].imshow(processed_image, cmap='gray', origin='lower')
        axes[i].set_title(f"File: {os.path.basename(file_path)}")
        axes[i].axis("off")

    # Turn off blank cells if the number of images is less than the number of cells
    for ax in axes[num_images:]:
        ax.axis("off")

    plt.tight_layout()
    plt.show()

# SHOW
DECRYPTED_DIR = "/content/drive/MyDrive/data-bigdatatechnlogy/BigData-project/decrypted_data/encrypted_ct_scans"
nii_files = [os.path.join(DECRYPTED_DIR, f) for f in os.listdir(DECRYPTED_DIR) if f.endswith('.nii')]

if nii_files:
    print(f"Hiển thị {len(nii_files)} ảnh đầu tiên:")
    # Window width and center for soft tissue
    show_multiple_images_v2(nii_files, window_center=40, window_width=400)
else:
    print("Không có file NIfTI nào trong thư mục.")


In [ ]:
import os
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np

DECRYPTED_DIR = "/content/drive/MyDrive/data-bigdatatechnlogy/BigData-project/decrypted_data/encrypted_lung_mask"

# Arrange NIFTI files in natural order
def natural_sort_key(s):
    return [int(text) if text.isdigit() else text for text in s.split('_')]

nii_files = sorted([f for f in os.listdir(DECRYPTED_DIR) if f.endswith('.nii')], key=natural_sort_key)

# Function to read and display NIFTI file
def display_lung_masks(file_list, folder_path, rows=2, cols=5):
    fig, axes = plt.subplots(rows, cols, figsize=(20, 10))
    axes = axes.ravel()

    for i in range(rows * cols):
        if i >= len(file_list):
            break
        file_path = os.path.join(folder_path, file_list[i])
        img = nib.load(file_path).get_fdata()

        # Select a slice in the middle of the 3D shape
        slice_idx = img.shape[2] // 2
        mask_slice = img[:, :, slice_idx]

        # show images in colormap gray
        axes[i].imshow(mask_slice, cmap='gray')
        axes[i].set_title(f"File: {file_list[i]}")
        axes[i].axis('off')

    plt.tight_layout()
    plt.show()

# Show images
if nii_files:
    print(f"Hiển thị {min(20, len(nii_files))} ảnh đầu tiên:")
    display_lung_masks(nii_files, DECRYPTED_DIR, rows=2, cols=5)
else:
    print("Không có file NIFTI nào trong thư mục.")


In [ ]:
import os
import nibabel as nib
import matplotlib.pyplot as plt

DECRYPTED_DIR = "/content/drive/MyDrive/data-bigdatatechnlogy/BigData-project/decrypted_data/encrypted_infection_mask"

# List of decrypted .nii files
nii_files = [os.path.join(DECRYPTED_DIR, f) for f in os.listdir(DECRYPTED_DIR) if f.endswith('.nii')]

def show_slices(image_data, num_slices=3):
    """
      Display slices of the 3D image.
      :param image_data: 3D image as NumPy array.
      :param num_slices: Number of slices to display.
    """
    mid_slices = [dim // 2 for dim in image_data.shape]  # Get the midpoint of each axis
    fig, axes = plt.subplots(1, num_slices, figsize=(15, 5))

    for i, axis in enumerate(range(num_slices)):
        if axis == 0:
            slice_data = image_data[mid_slices[0], :, :]  # X-axis slice
            axes[i].imshow(slice_data.T, cmap="gray", origin="lower")
            axes[i].set_title("Slice along X-axis")
        elif axis == 1:
            slice_data = image_data[:, mid_slices[1], :]  # Y-axis slice
            axes[i].imshow(slice_data.T, cmap="gray", origin="lower")
            axes[i].set_title("Slice along Y-axis")
        elif axis == 2:
            slice_data = image_data[:, :, mid_slices[2]]  # Z-axis slice
            axes[i].imshow(slice_data.T, cmap="gray", origin="lower")
            axes[i].set_title("Slice along Z-axis")

        axes[i].axis("off")
    plt.show()

# Show images
for nii_file in nii_files[:20]:  # 20 files
    img = nib.load(nii_file)  # loading file .nii
    data = img.get_fdata()    # Get image data as NumPy array
    print(f"Hiển thị lát cắt của file: {nii_file}")
    show_slices(data)


In [ ]:
import os
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np

DECRYPTED_DIR = "/content/drive/MyDrive/data-bigdatatechnlogy/BigData-project/decrypted_data/encrypted_lung_and_infection_mask"

def natural_sort_key(s):
    return [int(text) if text.isdigit() else text for text in s.split('_')]

nii_files = sorted([f for f in os.listdir(DECRYPTED_DIR) if f.endswith('.nii')], key=natural_sort_key)

# Hàm để đọc và hiển thị file NIFTI
def display_lung_masks(file_list, folder_path, rows=2, cols=5):
    fig, axes = plt.subplots(rows, cols, figsize=(20, 10))
    axes = axes.ravel()

    for i in range(rows * cols):
        if i >= len(file_list):
            break
        file_path = os.path.join(folder_path, file_list[i])
        img = nib.load(file_path).get_fdata()

        # Select a slice in the middle of the 3D shape
        slice_idx = img.shape[2] // 2
        mask_slice = img[:, :, slice_idx]

        # display images with colormap gray
        axes[i].imshow(mask_slice, cmap='gray')
        axes[i].set_title(f"File: {file_list[i]}")
        axes[i].axis('off')

    plt.tight_layout()
    plt.show()

# SHOW
if nii_files:
    print(f"Hiển thị {min(20, len(nii_files))} ảnh đầu tiên:")
    display_lung_masks(nii_files, DECRYPTED_DIR, rows=2, cols=5)
else:
    print("Không có file NIFTI nào trong thư mục.")


In [ ]:
import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import os


def show_infection_mask(file_path):
    """
    Hiển thị infection mask với màu sắc chuẩn y học.
    """
    try:
        # Load ảnh từ file NIfTI
        img = nib.load(file_path)
        data = img.get_fdata()

        # Chọn lát cắt Axial giữa
        axial_slice_idx = data.shape[2] // 2
        axial_slice = data[:, :, axial_slice_idx]

        # Tạo màu sắc chuẩn y học (ví dụ: vùng nhiễm trùng màu đỏ)
        plt.imshow(axial_slice, cmap='Reds', origin='lower')  # Sử dụng cmap='Reds'

        plt.title(f"Infection Mask - Axial View\nFile: {os.path.basename(file_path)}")
        plt.axis("off")
        plt.colorbar(label="Infection Probability")  # Thêm colorbar
        plt.show()

    except Exception as e:
        print(f"Lỗi khi hiển thị ảnh: {e}")


# Thư mục chứa file infection mask đã giải mã
DECRYPTED_DIR = "/content/drive/MyDrive/data-bigdatatechnlogy/BigData-project/decrypted_data/encrypted_infection_mask"
nii_files = [os.path.join(DECRYPTED_DIR, f) for f in os.listdir(DECRYPTED_DIR) if f.endswith('.nii')]

# Hiển thị infection mask cho một vài file đầu tiên
for nii_file in nii_files[:5]: # Hiển thị 5 file đầu tiên
    print(f"Hiển thị infection mask cho file: {nii_file}")
    show_infection_mask(nii_file)